(pystan_schema)=
# Example of `InferenceData` schema in PyStan

The description of the `InferenceData` structure can be found {ref}`here <schema>`.

In [1]:
import arviz as az
import pystan
import pandas as pd
import numpy as np
import xarray
xarray.set_options(display_style="html");

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time_since_joined = data.time.values
slack_comments = data.comments.values
github_commits = data.commits.values
names = data.index.values
N = len(names)
data

,comments,commits,time
Alice,7500,25,4.5
Bob,10100,32,6.0
Cole,18600,49,7.0
Danielle,25200,66,12.0
Erika,27500,96,18.0


In [3]:
# data for out of sample predictions
candidate_devs = ["Francis", "Gerard"]
candidate_devs_time = np.array([3.6, 5.1])
N_pred = len(candidate_devs)

In [4]:
linreg_prior_code = """
data {
  int<lower=0> N;
  real time_since_joined[N];
}

generated quantities {
    real b0;
    real b1;
    real log_b_sigma;
    real<lower=0> b_sigma;
    
    real c0;
    real c1;
    real log_c_sigma;
    real<lower=0> c_sigma;
    
    vector[N] slack_comments_hat;
    vector[N] github_commits_hat;
    
    b0 = normal_rng(0,200);
    b1 = normal_rng(0,200);
    b_sigma = abs(normal_rng(0,300));
    log_b_sigma = log(b_sigma);
    
    c0 = normal_rng(0,10);
    c1 = normal_rng(0,10);
    c_sigma = fabs(normal_rng(0,6));
    log_c_sigma = log(b_sigma);
    
    for (n in 1:N) {
        slack_comments_hat[n] = normal_rng(b0 + b1 * time_since_joined[n], b_sigma);
        github_commits_hat[n] = normal_rng(c0 + c1 * time_since_joined[n], c_sigma);
    }
}
"""
sm_prior = pystan.StanModel(model_code=linreg_prior_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0d5510d8a96e25d9a54b4530edc08cfc NOW.


In [5]:
linreg_prior_data_dict = {"N": N, "time_since_joined": time_since_joined}
prior = sm_prior.sampling(data=linreg_prior_data_dict, iter=150, chains=1, algorithm='Fixed_param', warmup=0)

In [6]:
linreg_code = """
data {
  int<lower=0> N;
  vector<lower=0>[N] time_since_joined;
  vector<lower=0>[N] slack_comments;
  vector<lower=0>[N] github_commits;
  
  
  // out of sample prediction
  int<lower=0> N_pred;
  vector<lower=0>[N_pred] time_since_joined_pred;
}

parameters {
  real b0;
  real b1;
  real log_b_sigma;
  
  real c0;
  real c1;
  real log_c_sigma;
}

transformed parameters {
  real<lower=0> b_sigma = exp(log_b_sigma);
  real<lower=0> c_sigma = exp(log_c_sigma);
}

model {
  b0 ~ normal(0,200);
  b1 ~ normal(0,200);
  b_sigma ~ normal(0,300);
  slack_comments ~ normal(b0 + b1 * time_since_joined, b_sigma);
  github_commits ~ normal(c0 + c1 * time_since_joined, c_sigma);
  
}

generated quantities {
    // elementwise log likelihood
    vector[N] log_likelihood_slack_comments;
    vector[N] log_likelihood_github_commits;
    
    // posterior predictive
    vector[N] slack_comments_hat;
    vector[N] github_commits_hat;
    
    // out of sample prediction
    vector[N_pred] slack_comments_pred;
    vector[N_pred] github_commits_pred;
    
    // posterior predictive
    for (n in 1:N) {
        log_likelihood_slack_comments[n] = normal_lpdf(slack_comments[n] | b0 + b1 * time_since_joined[n], b_sigma);
        slack_comments_hat[n] = normal_rng(b0 + b1 * time_since_joined[n], b_sigma);
        
        log_likelihood_github_commits[n] = normal_lpdf(github_commits[n] | c0 + c1 * time_since_joined[n], c_sigma);
        github_commits_hat[n] = normal_rng(c0 + c1 * time_since_joined[n], c_sigma);
    }
    
    // out of sample prediction
    for (n in 1:N_pred) {
        slack_comments_pred[n] = normal_rng(b0 + b1 * time_since_joined_pred[n], b_sigma);
        github_commits_pred[n] = normal_rng(c0 + c1 * time_since_joined_pred[n], c_sigma);
    }
}
"""
sm = pystan.StanModel(model_code=linreg_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_bb366f8ad241e32840c67395e3324113 NOW.


In [7]:
linreg_data_dict = {"N": N, "slack_comments": slack_comments, "github_commits": github_commits, "time_since_joined": time_since_joined, "N_pred" : N_pred, "time_since_joined_pred" : candidate_devs_time}
posterior = sm.sampling(data=linreg_data_dict, iter=200, chains=4)

In [8]:
idata_stan = az.from_pystan(
    posterior=posterior,
    prior=prior,
    posterior_predictive=["slack_comments_hat","github_commits_hat"],
    prior_predictive=["slack_comments_hat","github_commits_hat"],
    observed_data=["slack_comments","github_commits"],
    constant_data=["time_since_joined"],
    log_likelihood={
        "slack_comments": "log_likelihood_slack_comments",
        "github_commits": "log_likelihood_github_commits"
    },
    predictions=["slack_comments_pred", "github_commits_pred"],
    predictions_constant_data=["time_since_joined_pred"],
    coords={"developer": names, "candidate developer" : candidate_devs},
    dims={
        "slack_comments": ["developer"],
        "github_commits" : ["developer"],
        "slack_comments_hat": ["developer"],
        "github_commits_hat": ["developer"],
        "time_since_joined": ["developer"],
        "slack_comments_pred" : ["candidate developer"],
        "github_commits_pred" : ["candidate developer"],
        "time_since_joined_pred" : ["candidate developer"],
    }
)

In [9]:
idata_stan

Inference data with groups:
	> posterior
	> sample_stats
	> log_likelihood
	> posterior_predictive
	> observed_data
	> constant_data
	> prior
	> sample_stats_prior
	> prior_predictive
	> predictions
	> predictions_constant_data

In this example, each variable has as dimensions a combination of the following three: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values.

* In the case of `chain` and `draw` it is an integer identifier starting at `0`.
* In the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [10]:
idata_stan.posterior

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 100)
Coordinates:
  * chain        (chain) int64 0 1 2 3
  * draw         (draw) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    b0           (chain, draw) float64 350.3 354.9 371.6 ... -112.1 -82.22
    b1           (chain, draw) float64 1.477e+03 1.474e+03 ... 1.538e+03
    log_b_sigma  (chain, draw) float64 7.42 7.421 7.377 ... 7.521 7.435 7.36
    c0           (chain, draw) float64 3.981 -3.339 12.62 ... 1.39 -8.23 19.64
    c1           (chain, draw) float64 4.955 6.046 4.439 ... 5.402 6.263 3.965
    log_c_sigma  (chain, draw) float64 1.929 1.511 2.26 ... 1.24 2.223 1.773
    b_sigma      (chain, draw) float64 1.67e+03 1.671e+03 ... 1.571e+03
    c_sigma      (chain, draw) float64 6.886 4.53 9.579 ... 3.457 9.237 5.886
Attributes:
    created_at:                 2020-02-06T23:13:09.338879
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [11]:
idata_stan.sample_stats

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 100)
Coordinates:
  * chain        (chain) int64 0 1 2 3
  * draw         (draw) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    accept_stat  (chain, draw) float64 0.9988 0.999 0.9589 ... 0.9973 0.9706
    stepsize     (chain, draw) float64 0.06717 0.06717 ... 0.05834 0.05834
    treedepth    (chain, draw) int64 3 8 9 10 10 8 10 10 ... 5 10 6 10 9 4 8 10
    n_leapfrog   (chain, draw) int64 15 255 847 1023 1023 ... 943 23 255 1023
    diverging    (chain, draw) bool False False False ... False False False
    energy       (chain, draw) float64 115.6 114.8 117.0 ... 122.4 116.3 118.3
    lp           (chain, draw) float64 -112.9 -114.5 -114.8 ... -114.7 -116.0
Attributes:
    created_at:                 2020-02-06T23:13:09.342773
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [12]:
idata_stan.log_likelihood

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 8 ... 92 93 94 95 96 97 98 99
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 -8.385 -8.481 ... -8.281
    github_commits  (chain, draw, developer) float64 -2.866 -2.879 ... -3.05
Attributes:
    created_at:                 2020-02-06T23:13:09.345563
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [13]:
idata_stan.posterior_predictive

<xarray.Dataset>
Dimensions:             (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain               (chain) int64 0 1 2 3
  * draw                (draw) int64 0 1 2 3 4 5 6 7 ... 92 93 94 95 96 97 98 99
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 7.36e+03 ... 2.889e+04
    github_commits_hat  (chain, draw, developer) float64 34.04 33.66 ... 87.77
Attributes:
    created_at:                 2020-02-06T23:13:09.347718
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [14]:
idata_stan.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) int64 7500 10100 18600 25200 27500
    github_commits  (developer) int64 25 32 49 66 96
Attributes:
    created_at:                 2020-02-06T23:13:09.330834
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [15]:
idata_stan.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2020-02-06T23:13:09.333142
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [16]:
idata_stan.prior

<xarray.Dataset>
Dimensions:      (chain: 1, draw: 150)
Coordinates:
  * chain        (chain) int64 0
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
Data variables:
    b0           (chain, draw) float64 -145.7 -285.1 290.2 ... -109.0 147.1
    b1           (chain, draw) float64 89.3 -230.2 -149.0 ... -103.5 309.6
    log_b_sigma  (chain, draw) float64 5.406 6.253 4.959 ... 5.387 4.735 5.409
    b_sigma      (chain, draw) float64 222.7 519.5 142.4 ... 218.5 113.9 223.5
    c0           (chain, draw) float64 17.01 -12.05 4.21 ... -12.06 0.4713 5.922
    c1           (chain, draw) float64 20.72 20.89 13.13 ... -6.914 -6.757 4.829
    log_c_sigma  (chain, draw) float64 5.406 6.253 4.959 ... 5.387 4.735 5.409
    c_sigma      (chain, draw) float64 3.731 6.33 4.664 ... 6.427 8.585 3.369
Attributes:
    created_at:                 2020-02-06T23:13:09.353739
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [17]:
idata_stan.sample_stats_prior

<xarray.Dataset>
Dimensions:      (chain: 1, draw: 150)
Coordinates:
  * chain        (chain) int64 0
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
Data variables:
    accept_stat  (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    created_at:                 2020-02-06T23:13:09.356957
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [18]:
idata_stan.prior_predictive

<xarray.Dataset>
Dimensions:             (chain: 1, developer: 5, draw: 150)
Coordinates:
  * chain               (chain) int64 0
  * draw                (draw) int64 0 1 2 3 4 5 6 ... 144 145 146 147 148 149
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 275.9 ... 5.859e+03
    github_commits_hat  (chain, draw, developer) float64 116.6 146.3 ... 87.24
Attributes:
    created_at:                 2020-02-06T23:13:09.359003
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [19]:
idata_stan.predictions

<xarray.Dataset>
Dimensions:              (candidate developer: 2, chain: 4, draw: 100)
Coordinates:
  * chain                (chain) int64 0 1 2 3
  * draw                 (draw) int64 0 1 2 3 4 5 6 7 ... 93 94 95 96 97 98 99
  * candidate developer  (candidate developer) <U7 'Francis' 'Gerard'
Data variables:
    slack_comments_pred  (chain, draw, candidate developer) float64 7.217e+03 ... 6.665e+03
    github_commits_pred  (chain, draw, candidate developer) float64 18.37 ... 42.34
Attributes:
    created_at:                 2020-02-06T23:13:09.349962
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [20]:
idata_stan.predictions_constant_data

<xarray.Dataset>
Dimensions:                 (candidate developer: 2)
Coordinates:
  * candidate developer     (candidate developer) <U7 'Francis' 'Gerard'
Data variables:
    time_since_joined_pred  (candidate developer) float64 3.6 5.1
Attributes:
    created_at:                 2020-02-06T23:13:09.334048
    inference_library:          pystan
    inference_library_version:  2.19.1.1